In [1]:
import pandas as pd
import json

In [2]:
# Topic/Employee Admin Parameters/sheet columns
adminList = ["TopicGoals", "EmpRoster", "EmpAffinity"]

adminCols = {"TopicGoals": ["Topic","Goal","std_hours"], 
             "EmpRoster": ["EmpNum","Last","First","Title","Department"], 
             "EmpAffinity": ["EmpNum","Last","First","Title","Topic","SkillRating_1to5"]}

# Timesheet parameters/sheet columns
TopicList = ["CAPA",
"Inspection",
"New Product",
"Audit",
"Training"
]

TopicCols = {
    "CAPA": ["IssueNum","TaskType","EmpNum","Hours","Date","Status","Product"],
"Inspection": ["InspectionNum","Product","InspType","EmpNum","Hours","Date","Status"],
"New Product": ["ProjectNum", "Product","TaskType","EmpNum","Hours","Date","Status"],
"Audit": ["AuditNo","Lead","Duration","TargetDate","ActualDate","Type","Title"],
"Training": ["Instructor","Duration","TargetDate","ActualDate","Type","DocNo","DocName","RefNum"]
}

sheets = adminList + TopicList
colLookups = {**adminCols, **TopicCols}

# desiredCols = [Topic, RefNum, Detail, SubType, Date, Hours, Status, EmpNum, EmpName]
renameDict ={
        "CAPA": {"IssueNum":"RefNum",
                 "Product": "Detail",
                 "TaskType": "SubType",
                 "EmpNum": "EmpNum",
                 "Hours": "Hours",
                 "Date": "Date",
                 "Status": "Status"},
"Inspection": {"InspectionNum": "RefNum",
               "Product": "Detail",
               "InspType": "SubType",
               "EmpNum": "EmpNum",
               "Hours": "Hours",
               "Date": "Date",
               "Status": "Status"},
"New Product": {"ProjectNum":"RefNum",
                "Product": "Detail",
                "TaskType": "SubType",
                "EmpNum": "EmpNum",
                "Hours": "Hours",
                "Date": "Date",
                "Status": "Status"},
"Audit": {"AuditNo": "RefNum",
          "Lead": "EmpNum",
          "Duration": "Hours",
#           "TargetDate",
          "ActualDate": "Date",
          "Type": "SubType",
         "Title": "Detail"},
"Training": {"Instructor": "EmpNum",
             "Duration": "Hours",
#              "TargetDate",
             "ActualDate": "Date",
             "Type": "SubType",
             "DocNo": "Detail",
#              "DocName"
             "RefNum": "RefNum"
            }
}

In [3]:
xlsx = pd.ExcelFile('TimeSheetGenerator.xlsx')

In [4]:
# QUALITY CHECK: Are expected sheets and columns present?
sheetCheck = {}
for sheet in sheets:
    if sheet in xlsx.sheet_names:
        sheetCheck[sheet] = sheet + " is present."
    else:
        sheetCheck[sheet] = "Error: "+ sheet + " NOT is present."

In [5]:
sheetCheck

{'TopicGoals': 'TopicGoals is present.',
 'EmpRoster': 'EmpRoster is present.',
 'EmpAffinity': 'EmpAffinity is present.',
 'CAPA': 'CAPA is present.',
 'Inspection': 'Inspection is present.',
 'New Product': 'New Product is present.',
 'Audit': 'Audit is present.',
 'Training': 'Training is present.'}

In [6]:
colCheck = {}
for sheet in sheets:
    checksheet = xlsx.parse(sheet_name=sheet)
    for col in colLookups[sheet]:
        if col not in checksheet.columns:
            colCheck[sheet] = "Error: Column "+ col + " is NOT present."
        else:
            colCheck[sheet] = "All columns present."

In [7]:
colCheck

{'TopicGoals': 'All columns present.',
 'EmpRoster': 'All columns present.',
 'EmpAffinity': 'All columns present.',
 'CAPA': 'All columns present.',
 'Inspection': 'All columns present.',
 'New Product': 'All columns present.',
 'Audit': 'All columns present.',
 'Training': 'All columns present.'}

In [8]:
# Create dictionary of dataframes with only needed columns from colLookups
dfDict = {}
for sheet in sheets:
    dfDict[sheet]=pd.read_excel(xlsx,sheet)[colLookups[sheet]]
    

In [9]:
# Add Status to Audit and Training for On-time/Late
for topic in ["Audit","Training"]:
    StatusList = []
    for index, row in dfDict[topic].iterrows():
        if row["ActualDate"] > row["TargetDate"]:
            StatusList.append("Late")
        else:
            StatusList.append("On Time")
    dfDict[topic]["Status"] = StatusList

In [10]:
# Standardize column names
for topic in TopicList:
#     print(topic)
    dfDict[topic]=dfDict[topic].rename(columns = renameDict[topic])
    dfDict[topic]["Topic"] = topic
    dfDict[topic] = dfDict[topic][["Topic", "RefNum", "Detail", "SubType", 
                                  "Date", "Hours", "Status", "EmpNum"]].astype({"EmpNum":"str"})

In [11]:
# combine all dfs
fulldf = pd.concat([dfDict[topic] for topic in TopicList])
# fulldf

In [12]:
####### SANKEY #######

In [35]:
# Restructure with needed info for Sankey
Sankey_df = fulldf.groupby(["EmpNum","Topic","Status"])["Hours"].agg(['count']).reset_index()
Sankey_df

,EmpNum,Topic,Status,count
0,29436,CAPA,Late,3
1,29436,CAPA,On Time,1
2,29436,Inspection,Late,3
3,29436,Inspection,On Time,3
4,29436,Training,On Time,7
5,29436.0,New Product,Late,4
6,29436.0,New Product,On Time,3
7,32301,Audit,On Time,4
8,32301,CAPA,Late,1
9,32301,CAPA,On Time,2


In [14]:
# Create Sankey data with records structure
ontimeSankey = Sankey_df.to_dict(orient="records")
# ontimeSankey

In [15]:
## Radar data##

In [25]:
people_df= dfDict["EmpAffinity"][["EmpNum","Title","Topic","SkillRating_1to5"]].astype({"EmpNum":"str"})
# people_df

In [28]:
# set up lookups for name and title
dfDict["EmpRoster"]["EmpName"] = dfDict["EmpRoster"]["First"].str[0]+" "+dfDict["EmpRoster"]["Last"]
infodict = dfDict["EmpRoster"][["EmpNum","EmpName","Title"]].astype({"EmpNum":"str"}).set_index("EmpNum").to_dict(orient="index")

# set up affinity and hours spent on topic with structure needed for radar chart
radardict = {}
for person in people_df["EmpNum"].unique():
    affinity = people_df[people_df["EmpNum"]==person][["Topic","SkillRating_1to5"]].set_index("Topic").to_dict()["SkillRating_1to5"]
    TopicHours = fulldf[fulldf["EmpNum"]==person][["Topic","Hours"]].groupby("Topic").sum("Hours").to_dict()["Hours"]
    radardict[person]={}
    radardict[person]["affinity"] = affinity
    radardict[person]["topicHours"]=TopicHours
    radardict[person]["name"] = infodict[person]["EmpName"]
    radardict[person]["title"] = infodict[person]["Title"]

In [18]:
#### Add in Goal Levels

In [29]:
topicGoals = dfDict["TopicGoals"].set_index("Topic").to_dict(orient="index")

In [30]:
topicGoals

{'CAPA': {'Goal': 0.2, 'std_hours': 8},
 'Inspection': {'Goal': 0.25, 'std_hours': 10},
 'New Product': {'Goal': 0.3, 'std_hours': 12},
 'Audit': {'Goal': 0.15, 'std_hours': 6},
 'Training': {'Goal': 0.1, 'std_hours': 4}}

In [21]:
### Create json structure

In [31]:
outdata = {
    "Goals": topicGoals,
    "radarData": radardict,
    "sankeyData": ontimeSankey
}

In [32]:
outdata

{'Goals': {'CAPA': {'Goal': 0.2, 'std_hours': 8},
  'Inspection': {'Goal': 0.25, 'std_hours': 10},
  'New Product': {'Goal': 0.3, 'std_hours': 12},
  'Audit': {'Goal': 0.15, 'std_hours': 6},
  'Training': {'Goal': 0.1, 'std_hours': 4}},
 'radarData': {'29436': {'affinity': {'Training': 2,
    'CAPA': 2,
    'Inspection': 4,
    'New Product': 1,
    'Audit': 3},
   'topicHours': {'CAPA': 11.0, 'Inspection': 7.5, 'Training': 6.0},
   'name': 'K Hart',
   'title': 'Inspector'},
  '32301': {'affinity': {'CAPA': 5,
    'Inspection': 4,
    'New Product': 4,
    'Audit': 4,
    'Training': 5},
   'topicHours': {'Audit': 10.5,
    'CAPA': 8.0,
    'Inspection': 3.75,
    'Training': 6.25},
   'name': 'J Scott',
   'title': 'Quality Supervisor'},
  '49570': {'affinity': {'Inspection': 4,
    'New Product': 1,
    'Audit': 4,
    'Training': 2,
    'CAPA': 3},
   'topicHours': {'Audit': 20.5,
    'CAPA': 21.0,
    'Inspection': 6.75,
    'Training': 2.0},
   'name': 'Z Taylor',
   'title': 'In

In [34]:
json_string = json.dumps(outdata)
# print(json_string)
with open('json_data.json', 'w') as outfile:
    outfile.write(json_string)